# Tutorial-03 Intro to lateral Stanley control 

控制是整个无人平台系统的最底层，接受规划的路径和速度信息并执行。

<img src="https://qrq-1305568025.cos.ap-nanjing.myqcloud.com/obsidian_figs/general_view.jpg" width="500" height="350">

控制又可以进一步细分为横向控制和纵向控制，对应于实际驾驶过程中的踩油门/刹车和打方向盘的操作，后续的课程将会自底向上的尝试控制和规划部分，有时间的话会介绍感知和定位。

Stanley 横向控制算法的名称来源于 2005 年 DARPA 挑战赛斯坦福大学冠军车队的名字 "Stanley"，是一种基于横向跟踪误差为前轴中心到最近路径点的距离的非线性反馈函数，并且能实现横向跟踪误差指数收敛于0。根据车辆位姿与给定路径的相对几何关系可以直观的获得控制车辆方向盘转角的控制变量。

前轮转角控制变量由两部分构成: 一部分是航向误差引起的转角，即当前车身方向与参考轨迹最近点的切线方向的夹角;另一部分是横向误差引起的转角，即前轮中心到参考轨迹最近点的横向距离。

<img src="./media/stanley.png" width="420" height="525">

我们定义距离车辆前轮中心最近的点为参考点 $P_1$，$P_1$ 点切线的前方 $d(t)$ 距离处存在一个 $P_2$ 点，Stanley 方法的基本思路就是减小横向误差 $e_y$ 和航向误差 $\theta_y$ 

在**不考虑横向误差**的情况下，前轮偏角应当与给定路径参考点的切线方向一致。其中，$\theta_\varphi$ 表示车辆航向与最近路径点切线方向之间的夹角，在没有任何横向误差的情况下，前轮转角度 $\delta_\varphi$ 方向与所在路径点的方向相同。
$$
\delta_\varphi = \theta_\varphi
$$

在**不考虑航向误差**的情况下，横向跟踪误差越大，前轮转向角越大，假设车辆预期轨迹在距离前轮d(t)处与给定路径上最近点切线相交，根据几何关系得出如下非线性比例函数:
$$
\delta_y = \theta_y = \arctan \frac{e_y(t)}{d(t)} = \arctan \frac{ke_y(t)}{v(t)}
$$
其中，$d(t)$ 处与车速相关，用车速 $v(t)$ 和增益参数 $k$ 表示，$d(t) = ke_y(t)$.

综合横向误差和航向误差，可以得到控制量：
$$
\delta = \delta_\varphi + \delta_y
$$

式中有个重要的参量 -- 横向误差 $e(y)$，横向误差的正负决定了车辆相对于道路的位置，其计算方式可以由下图的几何关系确定：

<img src="./media/latError.png" width="470" height="525">

$$
\begin{align}
d_y &= y_r - y \\
d_x &= x_r - x \\
\vec{n}_r &= (-\sin\theta, \cos\theta) \\
e_y &= (\vec{X} - \vec{X}_r) \cdot \vec{n}_r \\
    &= d_y \cos \theta - d_x \sin\theta
\end{align}
$$


In [24]:
import math

'''
将角度归一化到 [-pi, pi]
'''
def normalize_angle(angle):
    normalized_angle = math.fmod(angle + math.pi, 2.0 * math.pi)
    
    if (normalized_angle < 0.0):
        normalized_angle = normalized_angle + 2.0 * math.pi
    
    return normalized_angle - math.pi

'''
创建一个存储坐标和航向的结构体
'''
class Transform:
    def __init__(self, x, y, theta):
        self.x_     = x
        self.y_     = y
        self.theta_ = normalize_angle(theta)

'''
创建一个存储车辆状态的结构体
'''
class State:
    def __init__(self, pose: Transform, v):
        self.pose_ = pose
        self.v_    = v
        
'''
创建 Stanley 控制器类
'''
class Stanley:
    def __init__(self, k):
        self.k_ = k # 初始化预瞄距离增益参数
        
    def update_control(self, P1: Transform, ego: State):
        # 计算横向误差及控制量
        # TODO
        dx = P1.x_ - ego.pose_.x_
        dy = P1.y_ - ego.pose_.y_
        ey= -dx * math.sin(ego.pose_.theta_) + dy * math.cos(ego.pose_.theta_)
        
        # 计算航向误差及控制量
        # TODO
        he= normalize_angle(P1.theta_ - ego.pose_.theta_)
        
        # 计算最终的转向角度
        # TODO
        theata_d = math.atan2(self.k_*ey, ego.v_+0.0001)
        delta = normalize_angle(theata_d+he)
        
        return delta 

和上节课一样，我们需要一个简单的车辆动力学模型来模拟车辆的运动过程，这里直接给出离散化的结果：
$$
\begin{align}
    x_{k+1} &= x_k + v_k \cos(\theta_k) \cdot dt \\
    y_{k+1} &= y_k + v_k \sin(\theta_k) \cdot dt \\
    \theta_{k+1} &= \theta_k + \frac{v_k}{L} \tan(\delta_k) \cdot dt \\
    v_{k+1} &= v_k + a_k \cdot dt
\end{align}
$$

其中：
- $x_k$ 和 $y_k$ 是在第 $k$ 个时间步的车辆位置
- $\theta_k$ 是在第 $k$ 个时间步的车辆航向，相对于x轴的正向角度。
- $v_k$ 是在第 $k$ 个时间步的车辆速度。
- $\delta_k$ 是在第 $k$ 个时间步的车辆转向角度，即车辆的前轮相对于车辆本身的转向角度。
- $a_k$ 是在第 $k$ 个时间步的车辆加速度。
- $L$ 是车辆的轴距，即前轮和后轮之间的距离。
- $dt$ 是时间步长。

In [25]:
'''
创建一个车辆运动学模型
'''
class Vehicle:
    def __init__(self, L, initial_state: State):
        self.L_ = L
        self.x_ = initial_state.pose_.x_
        self.y_ = initial_state.pose_.y_
        self.theta_ = normalize_angle(initial_state.pose_.theta_)
        self.v_ = initial_state.v_

    def update_state(self, delta, a, dt):
        # 更新车辆 x y theta v
        
        self.x_=self.x_+self.v_*math.cos(self.theta_)*dt
        self.y_=self.y_+self.v_*math.sin(self.theta_)*dt
        self.theta_=self.theta_+self.v_*math.tan(delta)/self.L_*dt
        self.v_=self.v_+a*dt
        new_pose  = Transform(self.x_, self.y_, self.theta_)
        new_state = State(new_pose, self.v_)
         
        return new_state

别忘了上次课的 PID 控制器

In [26]:
'''
创建 PID 控制器类
'''
class PidController:
    previous_error_ = 0 # 上一个控制周期的误差
    accum_error_    = 0 # 累计误差
    
    def __init__(self, Kp, Ki, Kd):
        self.Kp_ = Kp # 初始化比例系数
        self.Ki_ = Ki # 初始化积分系数
        self.Kd_ = Kd # 初始化微分系数
        
    def update_control(self, target_v, current_v, dt):
        # 计算比例项
        error = target_v - current_v
        proportional_term = self.Kp_ * error    

        
        # 计算积分项
        self.accum_error_ += error #* dt  # 累积误差
        integral_term = self.Ki_ * self.accum_error_

        
        # 计算微分项
        error_diff = error - self.previous_error_
        derivative_term = self.Kd_ * (error_diff / dt)

        
        # 计算控制量
        control_signal = proportional_term + integral_term + derivative_term

        self.previous_error_ = error


        # 更新返回值
        return control_signal

为了将结果清晰的展现出来，还需要一个绘制动画的函数

In [27]:
%matplotlib
import matplotlib.pyplot as plt

'''
绘制动画
'''
def animate(ref_x, ref_y, actual_x, actual_y, ego: State, L, area):
    actual_x.append(ego.pose_.x_)
    actual_y.append(ego.pose_.y_)
    
    plt.cla()
    
    plt.plot(ref_x, ref_y, label='reference path')
    
    half = len(actual_x) // 2
    plt.plot(actual_x[half:], actual_y[half:], label='history path')
    
    plt.xlim(ego.pose_.x_ - area, ego.pose_.x_ + area)
    plt.ylim(ego.pose_.y_ - area, ego.pose_.y_ + area)
    
    front_x = ego.pose_.x_ + L * math.cos(ego.pose_.theta_)
    front_y = ego.pose_.y_ + L * math.sin(ego.pose_.theta_)
    plt.plot([ego.pose_.x_, front_x], [ego.pose_.y_, front_y], label='vehicle body', linewidth=4)
    plt.scatter(ego.pose_.x_, ego.pose_.y_, label='reax axle center')
    
    plt.title("v[km/h]:" + str(ego.v_ * 3.6)[0:4])
    plt.grid(True)
    plt.legend()
    plt.box(True)
    plt.pause(0.01)

Using matplotlib backend: TkAgg


**Step 1** 通过 numpy 和 pandas 读取 csv 文件中存放的规划结果并通过 Matplotlib 显示出来


In [28]:
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

# 通过 numpy 的 genfromtxt 读取 csv 文件，并跳过第一行
trajectory_data = np.genfromtxt('./data/global_trajectory.csv', delimiter=';', skip_header=1)

# 为了避免行向量和列向量混淆导致的错误，尽可能使用 np.mat 而非 np.array 来进行数组操作
# 所以这里将读取的数据转换为了 np.mat 的格式
reference_trajectory = np.mat(trajectory_data)

# 将数组分割，方便后续操作
reference_s     = reference_trajectory[:, 0]             # [m]
reference_x     = reference_trajectory[:, 1]             # [m]
reference_y     = reference_trajectory[:, 2]             # [m]
reference_theta = reference_trajectory[:, 3] + math.pi/2 # [rad]
reference_v     = reference_trajectory[:, 5] * 0.44704   # [m/s]

# 使用 pyplot 的 Normalize 函数，将车速的最小值和最大值进行归一化处理
# 使其值位于 0-1 之间，从而绘制渐变的图形
norm = plt.Normalize(reference_v.min(), reference_v.max())

# 定义一个字典，用于描述色彩映射的过程。在这个字典中，键是颜色通道（'red', 'green', 'blue'），值是一个元组列表。
# 每个元组表示一个颜色通道在色彩映射中的变化过程，元组的第一个元素是颜色通道的位置（从 0 到 1），第二个和第三个元素是该位置的颜色值（也是从0到1）。
# 在此例中，我们定义了一个从蓝色过渡到红色的色彩映射。
cdict = {'red':   ((0.0, 0.0, 0.0), (1.0, 1.0, 1.0)), 
         'green': ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0)), 
         'blue':  ((0.0, 1.0, 1.0), (1.0, 0.0, 0.0))} 

# 使用定义好的颜色字典创建线性分段色彩映射，映射的名称为 'Bl_Rd'，分段数为 256
c_m = LinearSegmentedColormap('Rd_Bl', cdict, 256)

# 创建一个新的图窗，并设置其尺寸为 8x8
fig, ax = plt.subplots(figsize=(8, 8))

# 遍历参考路径中的每个点，根据车速来选取颜色，并绘制对应的路径
for i in range(1, len(reference_x)):
    x = np.array(reference_x[i-1:i+1]).ravel()
    y = np.array(reference_y[i-1:i+1]).ravel()
    c = c_m(norm(np.array(reference_v[i-1:i+1]).ravel()))
    ax.plot(x, y, color=c[0])

# 在右边放置一个 colorbar
sm = cm.ScalarMappable(cmap=c_m, norm=norm)
sm.set_array([])
fig.colorbar(sm, label='Speed [m/s]')
plt.grid(True)
plt.title('Global XY plot of reference path')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.axis('equal')
plt.savefig('./figs/global_trajectory.png')
plt.close()


**Step 2** 在循环中更新车辆位置并控制车辆行驶

In [29]:
import numpy as np
from scipy.spatial import cKDTree

# 初始化 KD 树
reference_points = np.hstack((reference_x, reference_y))
kd_tree = cKDTree(reference_points)

# 初始化车辆
initial_v     = reference_v[0, 0]
initial_x     = reference_x[0, 0]
initial_y     = reference_y[0, 0]
initial_theta = reference_theta[0, 0]
wheelbase     = 2

initial_state = State(Transform(initial_x, initial_y, initial_theta), initial_v)
vehicle = Vehicle(wheelbase, initial_state)

# 初始化 PID 控制器
pid = PidController(Kp=1.0, Ki=0.1, Kd=0.001)

# 初始化控制器
stanley = Stanley(k=1)

# 设置仿真条件
total_sim_time = 100
sample_time    = 0.1
sim_time       = np.arange(0, total_sim_time, sample_time)

# 用于记录数据的空数组
actual_x_list     = []
actual_y_list     = []
actual_theta_list = []
actual_v_list     = []
actual_err_list   = []
actual_v_ref_list = []

# 开始仿真
target_idx = 0
current_state = initial_state
plt.figure(2, figsize=(8, 8))

for i in range(len(sim_time)):
    plt.figure(2, figsize=(8, 8))
    animate(reference_x, reference_y, actual_x_list, actual_y_list, current_state, wheelbase, area=15)
    
    actual_x_list.append(current_state.pose_.x_)
    actual_y_list.append(current_state.pose_.y_)
    actual_theta_list.append(current_state.pose_.theta_)
    actual_v_list.append(current_state.v_)
    
    front_x = current_state.pose_.x_ + wheelbase * math.cos(current_state.pose_.theta_)
    front_y = current_state.pose_.y_ + wheelbase * math.sin(current_state.pose_.theta_)
    
    tracking_err, target_idx = kd_tree.query([front_x, front_y])
    
    target_x     = reference_x[target_idx, 0]
    target_y     = reference_y[target_idx, 0]
    target_theta = reference_theta[target_idx, 0]
    target_v     = reference_v[target_idx, 0]
    target_pose  = Transform(target_x, target_y, target_theta)
    
    actual_v_ref_list.append(target_v)
    actual_err_list.append(tracking_err)
    
    delta = stanley.update_control(target_pose, current_state)
    accel = pid.update_control(target_v, current_state.v_, sample_time)
    current_state = vehicle.update_state(delta, accel, sample_time)
    
print('Done!')

# 绘制结果
plt.figure(3, figsize=(8, 8))

# 创建第一个子图
plt.subplot(2, 1, 1)
plt.plot(sim_time, actual_v_ref_list, label='reference speed')
plt.plot(sim_time, actual_v_list, label='actual speed')
plt.ylabel('v [m/s]')
plt.axis('tight')
plt.grid(True)
plt.box(True)
plt.legend()

# 创建第二个子图
plt.subplot(2, 1, 2)
plt.plot(sim_time, actual_err_list)
plt.ylabel('lateral error [m/s]')
plt.xlabel('time [s]')
plt.axis('tight')
plt.grid(True)
plt.box(True)

# 显示图形
plt.show()

KeyboardInterrupt: 